## Homework 3 - Which book would you recommend?

-------------

### Importing related libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import lxml
import csv
import re
import os
import pandas as pd
from langdetect import detect
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer
import pandas as pd
import glob
import pickle
import functions

## 1. Data collection

### 1.1. Get the list of books

* From [best books ever list](https://www.goodreads.com/list/show/1.Best_Books_Ever?page=1) we want to collect the url associated to each book in the list and retrieve only the urls of the books listed in the first 300 pages.



* The output of this step is a `.txt` file whose single line corresponds to a book's url.

In [ ]:
#inizialize an empty list
url = []

#for each page, save the corresponding web page, find the anchor elements and save the corresponding tags  
for i in range(1, 301):
    page = requests.get("https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+ str(i))
    soup = BeautifulSoup(page.content, features='lxml')
    tag_a = soup.find_all('a', {"class": "bookTitle"}, itemprop = "url")
    
#for each book, save the corresponding url into the array
    for j in range(0,100):
        url.append("https://www.goodreads.com"+ tag_a[j]['href'])
        
#create a txt file where for each row there is a book's url 
with open("url.txt", 'w') as f:
    f.write("\n".join(map(str, url)))


### 1.2. Crawl books

1. Download the html corresponding to each of the collected urls.


2. After you collect a single page, immediatly save its `html` in a file. In this way, if your program stops, for any reason, you will not loose the data collected up to the stopping point.


3. Organize the entire set of downloaded `html` pages into folders. Each folder will contain the `htmls` of the books in page 1, page 2, ... of the list of books.

# Warning :

### Do not run the below cell because it'll download over 20 GB files !
#### Before running modify the `for` loop range to a lower number (e.g. 3) 

In [ ]:
#First we open the url.txt file, reading the lines and then close the file
f = open("url.txt", "r")
lines = f.readlines()
f.close()

#Setting our parent directory and the directory associated with each page number from the list
## PLEASE CHANGE THE PARENT DIRECTORY ACCORDING TO YOUR SYSTEM
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6/Links"
directory = "Page_"
page_num = 0


#Looping for 300 times that corresponds to the number of pages
for i in range(300):
    #Incrementing page number according to each page
    page_num += 1
    #Setting the current working directory
    directory = "Page_" + str(page_num)
    #Setting the main path to create the directory
    path = os.path.join(parent_dir, directory)
    #Creating new directory
    os.makedirs(path)
    
    #Looping for 100 times, which is the number of articles per page
    for i in range(100):
        #Selecting the corresponding link
        link = lines[i][:-1]
        #Dowloading the article
        r = requests.get(link, allow_redirects=True)
        #Setting the name file to keep the track
        file_name = "article_" + str(i+1) + ".html"
        #Saving the html file to its corresponding directory
        open(parent_dir + "/" + directory + "/" + file_name, 'wb').write(r.content)

-----------

#### 1.3 Parse downloaded pages

* Extracting the books information for each book as following:

1. Title (to save as `bookTitle`)
2. Series (to save as `bookSeries`)
3. Author(s), the first box in the picture below (to save as `bookAuthors`)
4. Ratings, average stars (to save as `ratingValue`)
5. Number of givent ratings (to save as `ratingCount`)
6. Number of reviews (to save as `reviewCount`)
7. The entire plot (to save as `Plot`)
8. Number of pages (to save as `NumberofPages`)
9. Published (Publishing Date)
10. Characters
11. Setting
12. Url

In [ ]:
## Custom function designed to 
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6"

functions.info_parser(parent_dir, pages=300, tsv_articles= "tsv_articles", links= "Links", url= 'url')

All tsv files generated sucessfully in tsv_articles directory

--------------

## 2. Search Engine

Now, we want to create two different Search Engines that, given as input a query, return the books that match the query.

First, you must pre-process all the information collected for each book by :



1. Removing stopwords
2. Removing punctuation
3. Stemming
4. Anything else you think it's needed


For this purpose, you can use the [nltk](https://www.nltk.org/) library.

* At first we want to load all the .tsv files into one .csv file

In [4]:
#Creating a .csv files out of all .tsv files
'''
parent_dir = "C:/Users/engme/OneDrive/Desktop/ALL Materials/Data Science - Sapienza/1st Semester/ALGORITHMIC METHODS OF DATA MINING AND LABORATORY/Labs/6"
tsv_folder = "tsv_articles"

'''
# The function to create .csv file
'''
df_tsv = functions.create_csv(parent_dir, tsv_folder, export_csv= False)

''';
#Reading the created .csv file
df_tsv = pd.read_csv("files/combined_csv.csv")

### 2.1. Conjunctive query

For the first version of the search engine, we narrow our interest on the `Plot` of each document. It means that you will evaluate queries only with respect to the book's plot.

**2.1.1) Create your index!**

Before building the index,
Create a file named `vocabulary`, in the format you prefer, that maps each word to an integer (`term_id`).

In [5]:
# Creating the inverted index and vocabulary dictionaries
'''
inv_index1, vocabulary, processed_docs = functions.create_dictionary_plot(df_tsv, export_json = True)

''';

# Loading the saved dictionaries
with open("files/dict_file", "rb") as input_file:
    inv_index1 = pickle.load(input_file)
    input_file.close
    
with open("files/voc_file", "rb") as input_file:
    vocabulary = pickle.load(input_file)
    input_file.close
    
with open("files/doc_file", "rb") as input_file:
    processed_docs = pickle.load(input_file)
    input_file.close

**2.1.2) Execute the query**

Given a query, that you let the user enter, the Search Engine is supposed to return a list of documents.

> What documents do we want?

Since we are dealing with conjunctive queries (AND), each of the returned documents should contain all the words in the query. The final output of the query must return, if present, the following information for each of the selected documents:

* `bookTitle`
* `Plot`
* `Url`

In [6]:
query = input("Please Enter your query: ")
functions.Search_Engine1(query, df_tsv, vocabulary, inv_index1, results= 20)

Please Enter your query: harry potter


,bookTitle,Plot,Url
22715,James Potter and the Vault of Destinies,This Changes Everything... It is a year of cha...,https://www.goodreads.com/book/show/5129.Brave...
11323,My Immortal,"Ebony, a self-proclaimed goth and a vampire, f...",https://www.goodreads.com/book/show/231804.The...
2776,Harry Potter and the Chamber of Secrets,Ever since Harry Potter had come home for the ...,https://www.goodreads.com/book/show/7144.Crime...
6893,Island of Graves,"On the brink of war, the fates of both Artimé ...",https://www.goodreads.com/book/show/30.J_R_R_T...
14658,Harry Potter and the Methods of Rationality,Harry Potter and the Methods of Rationality is...,https://www.goodreads.com/book/show/252577.Ang...
23999,Endless Magic,<div>\r\n<div>\r\nYOU'VE BEEN LOOKING FOR SOME...,https://www.goodreads.com/book/show/3590.The_A...
2016,Fantastic Beasts and Where to Find Them,An approved textbook at Hogwarts School of Wit...,https://www.goodreads.com/book/show/3263607-th...
6198,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ...",https://www.goodreads.com/book/show/968.The_Da...
18063,Queste,<strong>The fourth book in the internationally...,https://www.goodreads.com/book/show/7624.Lord_...
25673,House of Secrets,From legendary Hollywood director Chris Columb...,https://www.goodreads.com/book/show/3263607-th...


### 2.2) Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

* Find all the documents that contains all the words in the query.
* Sort them by their similarity with the query
* Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.

To solve this task, you will have to use the tfIdf score, and the Cosine similarity. The fielf to consider it is still the plot. Let's see how.

**2.2.1) Inverted index**

In [7]:
'''
inv_index2 = functions.create_invert_index2(len(df_tsv), inv_index1, processed_docs)
dictSimilarity = functions.create_similarity_dictionary(inv_index2)

''';


with open("files/dictsim_file", "rb") as input_file:
    dictSimilarity = pickle.load(input_file)
    input_file.close

with open("files/inv2_file", "rb") as input_file:
    inv_index2 = pickle.load(input_file)
    input_file.close

**2.2.2) Execute the query**

In this new setting, given a query you get the right set of documents (i.e., those containing all the words in the query) and sort them according to their similairty to the query. For this purpose, as scoring function we will use the Cosine Similarity with respect to the tfIdf representations of the documents.

The search engine is supposed to return a list of documents, ranked by their Cosine Similarity with respect to the query entered in input.

>More precisely, the output must contain:

* `bookTitle`
* `Plot`
* `Url`
* The similarity score of the documents with respect to the query

In [8]:
query = input("Enter your query: ")
functions.Search_Engine2(query, df_tsv, inv_index1, inv_index2, dictSimilarity, vocabulary, results= 20)

Enter your query: harry potter


,bookTitle,Plot,Url,Similarity
23166,The Harry Potter trilogy,This box set collects hard cover editions Har...,https://www.goodreads.com/book/show/1953.A_Tal...,1.00
23590,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",Box Set containing Harry Potter and the Sorcer...,https://www.goodreads.com/book/show/3636.The_G...,1.00
3220,Harry Potter and the Cursed Child: Parts One a...,Based on an original new story by J.K. Rowling...,https://www.goodreads.com/book/show/929.Memoir...,0.91
6198,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ...",https://www.goodreads.com/book/show/968.The_Da...,0.87
20482,Harry Potter Series Box Set,"Over 4000 pages of Harry Potter and his world,...",https://www.goodreads.com/book/show/386162.The...,0.76
14658,Harry Potter and the Methods of Rationality,Harry Potter and the Methods of Rationality is...,https://www.goodreads.com/book/show/252577.Ang...,0.63
323,Harry Potter and the Goblet of Fire,Harry Potter is midway through his training as...,https://www.goodreads.com/book/show/2657.To_Ki...,0.60
19511,Harry Potter and the Sorcerer's Stone,Harry Potter's life is miserable. His parents ...,https://www.goodreads.com/book/show/3.Harry_Po...,0.55
2776,Harry Potter and the Chamber of Secrets,Ever since Harry Potter had come home for the ...,https://www.goodreads.com/book/show/7144.Crime...,0.53
25775,Harry Potter and the Prisoner of Azkaban,"For twelve long years, the dread fortress of A...",https://www.goodreads.com/book/show/5.Harry_Po...,0.48


--------------------

## 3. Define a new score!

Now it's your turn. Build a new metric to rank books based on the queries of their users.

In this scenario, a single user can give in input more information than the single textual query, so you need to take into account all this information, and think a creative and logical way on how to answer at user's requests.

Practically:

The user will enter you a text query. As a starting point, get the query-related documents by exploiting the search engine of Step 3.1.

Once you have the documents, you need to sort them according to your new score. In this step you won't have anymore to take into account just the `plot` of the documents, you **must** use the remaining variables in your dataset (or new possible variables that you can create from the existing ones...). You **must** use a heap data structure (you can use Python libraries) for maintaining the *top-k* documents.

>Q: How to sort them? A: Allow the user to specify more information, that you find in the documents, and define a new metric that ranks the results based on the new request.

**N.B.**: You have to define a **scoring function**, not a filter!


The output, must contain:

* `bookTitle`
* `Plot`
* `Url`
* The similarity score of the documents with respect to the query

In [9]:
query = input("Enter your query: ")
functions.Search_Engine_NewScore(query, df_tsv, inv_index1, inv_index2, vocabulary, results= 20)

Enter your query: harry potter
Do you want to use our new scoring method? [Y/N] :y
How many pages does your desired book have? (approximately) ->850
What year does the book published? (approximately) ->2000


,bookTitle,Plot,Url,Similarity
book_index,,,,
162,The Young Wizards,Book club collection of So You Want to Be a W...,https://www.goodreads.com/book/show/2956.The_A...,0.96
23166,The Harry Potter trilogy,This box set collects hard cover editions Har...,https://www.goodreads.com/book/show/1953.A_Tal...,0.96
323,Harry Potter and the Goblet of Fire,Harry Potter is midway through his training as...,https://www.goodreads.com/book/show/2657.To_Ki...,0.93
2776,Harry Potter and the Chamber of Secrets,Ever since Harry Potter had come home for the ...,https://www.goodreads.com/book/show/7144.Crime...,0.89
1171,Harry Potter Schoolbooks Box Set: Two Classic ...,There is an alternate cover edition for this I...,https://www.goodreads.com/book/show/43763.Inte...,0.88
6198,The Harry Potter Collection 1-4,"The exciting tales of Harry Potter, the young ...",https://www.goodreads.com/book/show/968.The_Da...,0.87
2016,Fantastic Beasts and Where to Find Them,An approved textbook at Hogwarts School of Wit...,https://www.goodreads.com/book/show/3263607-th...,0.86
3381,Quidditch Through the Ages,The most checked-out book in the Hogwarts Libr...,https://www.goodreads.com/book/show/157993.The...,0.85
19511,Harry Potter and the Sorcerer's Stone,Harry Potter's life is miserable. His parents ...,https://www.goodreads.com/book/show/3.Harry_Po...,0.83
